# Groupchat with Llamaindex agents

(Llamaindex agents)[https://docs.llamaindex.ai/en/stable/optimizing/agentic_strategies/agentic_strategies/] have the ability to use planning strategies to answer user questions. They can be integrated in Autogen in easy ways

## Requirements

In [ ]:
! pip install pyautogen
! pip install llama-index
! pip install llama-index-tools-wikipedia
! pip install llama-index-readers-wikipedia
! pip install wikipedia

## Set your API Endpoint

In [2]:
import os

import autogen

config_list = [{"model": "gpt-3.5-turbo-0125", "api_key": os.getenv("OPENAI_API_KEY")}]

## Set Llamaindex

In [3]:
from llama_index.core import Settings
from llama_index.core.agent import ReActAgent
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.tools.wikipedia import WikipediaToolSpec

llm = OpenAI(
    model= "gpt-3.5-turbo-0125",
    temperature=0.0,
    api_key= os.environ.get("OPENAPI_API_KEY", ""),
    )

embed_model = OpenAIEmbedding(
                model= "text-embedding-ada-002",
                temperature=0.0,
                api_key= os.environ.get("OPENAPI_API_KEY", ""),
                )

Settings.llm = llm
Settings.embed_model = embed_model

# create a react agent to use wikipedia tool
wiki_spec = WikipediaToolSpec()
# Get the search wikipedia tool
wikipedia_tool = wiki_spec.to_tool_list()[1]

location_specialist = ReActAgent.from_tools(
    tools=[wikipedia_tool], 
    llm=llm,
    max_iterations=10,
    verbose=True)

## Create agents

In this example, we will create a Llamaindex agent to answer questions fecting data from wikipedia and a user proxy agent.

In [ ]:
from llamaindex_conversable_agent import LLamaIndexConversableAgent

llm_config = {
    "temperature": 0,
    "config_list": config_list,
}

trip_assistant = LLamaIndexConversableAgent(
    "trip_specialist",
    llama_index_agent=  location_specialist,
    system_message="You help customers finding more about places they would like to visit. You can use external resources to provide mroe details as you engage with the customer.",
    description="This agents helps customers discover locations to visit, things to do, and other details about a location. It can use external resources to provide more details. This agent helps in finding attractions, history and all that there si to know abotu a place",
)

user_proxy = autogen.UserProxyAgent(
    name="Admin",
    human_input_mode="ALWAYS",
    code_execution_config=False,
)

Next, let's set up our group chat.

In [3]:
groupchat = autogen.GroupChat(
    agents=[trip_assistant, user_proxy],
    messages=[],
    max_round=500,
    speaker_selection_method="round_robin",
    enable_clear_history=True,
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [5]:
chat_result = user_proxy.initiate_chat(
    manager,
    message="""
What can i find in Tokyo related to Hayao Miyazaki and its moveis like Spirited Away?.
""",
)

Admin (to chat_manager):


You will need to improve app in FastApi. For now, check out all the application files, try to understand it and wait for next instructions.


--------------------------------------------------------------------------------
Engineer (to chat_manager):

***** Suggested tool Call (call_SA61u9yCLhyXfd9NCV9TAIiM): list_dir *****
Arguments: 

{
  "directory": "./"
}
*************************************************************************

--------------------------------------------------------------------------------


Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION list_dir...
Admin (to chat_manager):

Admin (to chat_manager):

***** Response from calling tool "call_SA61u9yCLhyXfd9NCV9TAIiM" *****
[0, ["main.py", "__pycache__"]]
**********************************************************************

--------------------------------------------------------------------------------
Engineer (to chat_manager):

***** Suggested tool Call (call_IVJNPI12s4fCzysnWjExZjL2): see_file *****
Arguments: 

{
  "filename": "main.py"
}
*************************************************************************

--------------------------------------------------------------------------------


Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION see_file...
Admin (to chat_manager):

Admin (to chat_manager):

***** Response from calling tool "call_IVJNPI12s4fCzysnWjExZjL2" *****
[0, "1:from fastapi import FastAPI\n2:import yfinance as yf\n3:\n4:app = FastAPI()\n5:\n6:@app.get(\"/cdr_daily_spread\")\n7:async def calculate_daily_spread():\n8:    cdr = yf.Ticker(\"CDR.WA\")\n9:    today_data = cdr.history(period=\"1d\")\n10:    spread = ((today_data[\"High\"] - today_data[\"Low\"]) / today_data[\"Low\"]) * 100\n11:    return spread"]
**********************************************************************

--------------------------------------------------------------------------------
Engineer (to chat_manager):

The application is a FastAPI application with a single endpoint `/cdr_daily_spread`. This endpoint calculates the daily spread of the stock "CDR.WA" using the `yfinance` library. The spread is calculated as the percentage differen

...

Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION see_file...
Admin (to chat_manager):

Admin (to chat_manager):

***** Response from calling tool "call_4wPhGFFW8pt8B28nVhVvWztN" *****
[0, "1:import yfinance as yf\n2:\n3:def calculate_daily_spread(ticker):\n4:    stock = yf.Ticker(ticker)\n5:    today_data = stock.history(period=\"1d\")\n6:    spread = ((today_data[\"High\"] - today_data[\"Low\"]) / today_data[\"Low\"]) * 100\n7:    return spread"]
**********************************************************************

--------------------------------------------------------------------------------
Engineer (to chat_manager):

***** Suggested tool Call (call_DL0p37GzppRSmoqFaOOLSNaI): modify_code *****
Arguments: 

{
  "filename": "spread_calculation.py",
  "start_line": 3,
  "end_line": 7,
  "new_code": "def calculate_daily_spread(ticker):\n    stock = yf.Ticker(ticker)\n    today_data = stock.history(period=\"1d\")\n    spread = ((today_dat

Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION modify_code...
Admin (to chat_manager):

Admin (to chat_manager):

***** Response from calling tool "call_DL0p37GzppRSmoqFaOOLSNaI" *****
[0, "Code modified"]
**********************************************************************

--------------------------------------------------------------------------------
Engineer (to chat_manager):

The changes have been made successfully. Now, the `calculate_daily_spread` function in `spread_calculation.py` returns a single value instead of a pandas Series. This should resolve the ValueError you were encountering.

--------------------------------------------------------------------------------


Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit
